## Import

In [1]:
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup
# !pip -q install finance-datareader
import FinanceDataReader as fdr

# pip install pandas-datareader
# from pandas_datareader import data as web
# import yfinance as yf
import requests
import json
from pandas import json_normalize
import xml.etree.ElementTree as ET
import zipfile
from io import BytesIO

# ignore warnings
import warnings ; warnings.filterwarnings('ignore')

### 한국거래소(KRX)
- [종목] http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201020201
- [주식추이] http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201020201

In [2]:
# 사이트에서 직접 다운로드 받는다.
code_list = pd.read_csv('data/data_0700_20231014.csv', encoding='cp949',
                        usecols=['한글 종목명','단축코드','상장일','시장구분','주식종류','액면가','상장주식수'])
code_list.sample()

,단축코드,한글 종목명,상장일,시장구분,주식종류,액면가,상장주식수
1280,139990,아주스틸보통주,2021/08/20,KOSPI,보통주,500,26452189


### 금융위원회
주식시세, 기업 정보 등을 API로 제공한다.
- [주식시세] https://www.data.go.kr/iim/api/selectAPIAcountView.do
- [기업재무정보] https://www.data.go.kr/data/15043459/openapi.do
- [공시정보] https://www.data.go.kr/data/15059649/openapi.do

In [3]:
# API 키, 데이터수, 질의 시작일자, 질의 종료일자를 입력하면 해당 기간의 주식시세를 불러오는 함수를 정의한다.
def FSC_smc(KEY, NUMOFROWS, START, END):
    total = pd.DataFrame()
    for DATE in pd.Series(pd.date_range(str(START), str(END))).dt.date.astype(str).str.replace('-',''):
        url = f'https://apis.data.go.kr/1160100/service/GetStockSecuritiesInfoService/getStockPriceInfo?serviceKey={KEY}&numOfRows={NUMOFROWS}&basDt={DATE}'
        result = urlopen(url) 
        stock = BeautifulSoup(result, 'lxml-xml')
        stock = pd.DataFrame([i.text.split('\n')[1:-1] for i in stock.find_all("item")], 
                              columns= ["date", "stockCode", "_", "stockName", "market", "close", "_", "_", 
                                        "open", "high", "low", "volume", "amount", "totShare", "totAmount"])
        stock = stock[[i for i in stock.columns if i != "_"]]
        total = pd.concat([total, stock])
    return total

In [4]:
key = "본인의 인증키"
nrows = 10000
start, end = 20230901, 20230905

# 거래일자, 주식코드, 주식명, 시장, 종가, 시가, 최고가, 최저가, 거래량, 거래대금, 상장주식수, 시가총액
smc = FSC_smc(key, nrows, start, end)
display(smc)

,date,stockCode,stockName,market,close,open,high,low,volume,amount,totShare,totAmount
0,20230901,900110,이스트아시아홀딩스,KOSDAQ,141,144,145,141,4167947,594074068,291932050,41162419050
1,20230901,900270,헝셩그룹,KOSDAQ,252,253,256,251,800180,202022415,85682000,21591864000
2,20230901,900260,로스웰,KOSDAQ,843,865,877,820,250955,213137681,36031288,30374375784
3,20230901,900290,GRT,KOSDAQ,3390,3675,3685,3315,1147403,3991659490,67375000,228401250000
4,20230901,900300,오가닉티코스메틱,KOSDAQ,170,169,172,167,1390610,234505501,245263481,41694791770
...,...,...,...,...,...,...,...,...,...,...,...,...
2746,20230905,900120,씨케이에이치,KOSDAQ,143,143,145,141,159232,22707211,197716999,28273530857
2747,20230905,900250,크리스탈신소재,KOSDAQ,2680,2600,2720,2600,5229043,13970616190,95891039,256987984520
2748,20230905,900070,글로벌에스엠,KOSDAQ,660,668,668,654,122822,81138960,53743968,35471018880
2749,20230905,900140,엘브이엠씨홀딩스,KOSPI,2565,2505,2595,2490,1159046,2929312300,171588411,440124274215


## FinanceDataReader
주식, 암호화폐 등의 일별 시가, 최고가, 최저가, 종가, 거래량, 변동률을 제공한다.
- StockListing<br>
  주식 정보를 반환한다.<br>
  $\bullet$ KRX: 한국 거래소 상장 주식 정보<br>
  $\bullet$ NASDAQ: 나스닥 상장 주식 정보<br>
  $\bullet$ NYSE: NYSE 상장 주식 정보<br>
  $\bullet$ AMEX: 미국 증권 거래소 상장 주식 정보<br>
  $\bullet$ S&P500: S&P500에 해당되는 주식 정보
- DataReader<br>
  주식 코드나 지수, 비트코인 코드와 조회 일자를 입력하면 기간 내 주식코드의 일별 거래 통계를 반환한다.<br>
   $\bullet$ KS11: 코스피 지수<br>
   $\bullet$ KQ11: 코스닥 지수<br>
   $\bullet$ DJI: 다우 지수<br>
   $\bullet$ IXIC: 나스닥 지수<br>
   $\bullet$ US500: S&P500 지수<br>
   $\bullet$ USD/KRX: 원달러 환율<br>
   $\bullet$ USD/EUR: 달러당 유로화 환율<br>
   $\bullet$ CNY/KRX: 위안화 원화 환율<br>
   $\bullet$ BTC/USD: 비트코인 가격<br>
   $\bullet$ BTC/KRX: 비트코인 원화 가격<br>

In [5]:
# 주식명, 질의 시작날짜, 질의 종료날짜를 입력하면 해당 기간의 주식거래 데이터를 불러오는 함수를 정의한다.
def fdr_stock(NAME, START, END):
    code = fdr.StockListing('KRX').query('Name == @NAME')['Code'].values[0]
    return fdr.DataReader(code, START, END)

In [6]:
# 예제로 삼성전자 주식을 불러온다.
samsung = fdr_stock("삼성전자", '2020-01-01', '2023-01-01')
display(samsung)

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2020-01-02,55500,56000,55000,55200,12993228,-0.010753
2020-01-03,56000,56600,54900,55500,15422255,0.005435
2020-01-06,54900,55600,54600,55500,10278951,0.000000
2020-01-07,55700,56400,55600,55800,10009778,0.005405
2020-01-08,56200,57400,55900,56800,23501171,0.017921
...,...,...,...,...,...,...
2022-12-23,58200,58400,57700,58100,9829407,-0.016920
2022-12-26,58000,58100,57700,57900,6756411,-0.003442
2022-12-27,58000,58400,57900,58100,10667027,0.003454


## 한국은행
은행, 시장지수 등을 API로 제공한다.
- [항목] https://ecos.bok.or.kr/api/#/DevGuide/StatisticalCodeSearch
  <img src="첨부사진/ecos.png">

In [7]:
# API 키, 지수코드, 질의 시작날짜, 질의 종료날짜, 데이터 기간단위, 계정항목코드에 만족하는 데이터 반환 함수를 정의한다.
def Kbank(KEY, SCODE, START, END, FREQ, ICODE):
    url = f"https://ecos.bok.or.kr/api/StatisticSearch/{KEY}/json/kr/1/10000/{SCODE}/{FREQ}/{START}/{END}/{ICODE}"
    r = requests.get(url)
    ecos = json.loads(r.text)
    ecos = pd.json_normalize(ecos['StatisticSearch']['row'])
    # 시계열 데이터로 변경
    ecos['TIME'] = pd.to_datetime(ecos['TIME'], format='%Y%m%d')
    ecos.set_index('TIME', inplace=True)
    # 불필요한 데이터 제거
    ecos.drop(ecos.columns[ecos.notna().sum() <= 10], axis=1, inplace=True)
    return ecos

In [8]:
# 예시로 외국인 순매수를 추출한다.
key = "본인의 인증키"
statcode = '802Y001' # 주식시장
freq, start, end = 'D', '20100101', '20230920'
itemcode = '0030000' # 외국인 순매수

fnb = Kbank(key, statcode, start, end, freq, itemcode)
display(fnb.head())

,STAT_CODE,STAT_NAME,ITEM_CODE1,ITEM_NAME1,UNIT_NAME,DATA_VALUE
TIME,,,,,,
2010-01-04,802Y001,1.5.1.1. 주식시장(일),0030000,"외국인 순매수(주식시장, 잠정치)",억원,2390
2010-01-05,802Y001,1.5.1.1. 주식시장(일),0030000,"외국인 순매수(주식시장, 잠정치)",억원,4005
2010-01-06,802Y001,1.5.1.1. 주식시장(일),0030000,"외국인 순매수(주식시장, 잠정치)",억원,3517
2010-01-07,802Y001,1.5.1.1. 주식시장(일),0030000,"외국인 순매수(주식시장, 잠정치)",억원,2233
2010-01-08,802Y001,1.5.1.1. 주식시장(일),0030000,"외국인 순매수(주식시장, 잠정치)",억원,619


## DART
기업재무정보를 제공한다.
- https://opendart.fss.or.kr/guide/main.do?apiGrpCd=DS006

#### 1. 기업 종목 코드
- [API설명서] https://opendart.fss.or.kr/guide/detail.do?apiGrpCd=DS001&apiId=2019018

In [9]:
# 전체 기업 종목코드를 불러온다.
key = '본인의 인증키'
url = f'https://opendart.fss.or.kr/api/corpCode.xml?crtfc_key={key}'
res = requests.get(url)
z = zipfile.ZipFile(BytesIO(res.content))
data = z.read('CORPCODE.xml').decode('utf-8')

In [10]:
# Parse the XML data
root = ET.fromstring(data)

In [11]:
# Get data per elements
corporation = []

for i in root.findall('list'):
    corporation.append([i.find(x).text for x in ['corp_code', 'corp_name', 'stock_code']])

# Make DataFrame and Save
# 기업명, 기업코드, 주식코드
corporation = pd.DataFrame(corporation, columns = ['corp_code', 'corp_name', 'stock_code'])
corporation.drop(corporation.query('stock_code==" "').index, inplace=True)
display(corporation.sample(5))

,corp_code,corp_name,stock_code
99628,01016886,테크엔,308700
95421,01021639,나라셀라,405920
80745,01246742,테크트랜스,258050
100844,00144720,와이투솔루션,011690
8719,00225140,쌈지,033260


#### 2. 기업개황 정보
- [API설명서] https://opendart.fss.or.kr/guide/detail.do?apiGrpCd=DS001&apiId=2019002

In [12]:
# corp_code는 반드시 입력되어야 한다.
code = '00126380' 
url = f'https://opendart.fss.or.kr/api/company.json?crtfc_key={key}&corp_code={code}'
res = requests.get(url)
data = json.loads(res.text)

In [13]:
# 기업명, 기업코드, 주식명, 법인여부, 업종코드
samsung = pd.DataFrame.from_dict(data, orient='index').T[['corp_name','corp_code','stock_name','corp_cls','induty_code']]
display(samsung)

,corp_name,corp_code,stock_name,corp_cls,induty_code
0,삼성전자(주),00126380,삼성전자,Y,264


#### 3. 상장기업 단일회사 정보
- [API설명서] https://opendart.fss.or.kr/guide/detail.do?apiGrpCd=DS003&apiId=2019016

In [14]:
# corp_code, bsns_year, reprt_code는 필수로 입력되어야 한다.
code = '00126380'
year = 2023
report = 11012   # 1분기(11013, 0331), 반기(11012, 0630), 3분기(11014, 0931), 사업(11011, 1231)

url = "https://opendart.fss.or.kr/api/fnlttSinglAcnt.json"
res = requests.get(url, params={'crtfc_key': key, 'corp_code': code, 'bsns_year': year, 'reprt_code': report})
data = json.loads(res.text)

In [15]:
# 보고서코드, 년도, 기업코드, 주식코드, 계정명, 개별/연결 구분, 당기금액
single = json_normalize(data['list']) [['reprt_code','bsns_year','corp_code', 'stock_code', 'account_nm', 'fs_div', 'thstrm_amount']]

# 보고서별 공시일자를 생성한다.
# pd.to_datetime의 errors는 예외처리 옵션이다.
single['monday'] = single['reprt_code'].map({'11011':'1231','11012':'0630','11013':'0331','11014':'0931'})
single['date'] = pd.to_datetime(single['bsns_year']+single['monday'], format='%Y%m%d', errors='coerce')
single.drop(['reprt_code','bsns_year','monday'], axis=1, inplace=True)

# 당기금액의 특수문자 제거, 조단위 변경을 수행한다.
single['thstrm_amount'] = single['thstrm_amount'].str.replace(',', '').astype(float)/1000000000000
display(single.sample(5))

,corp_code,stock_code,account_nm,fs_div,thstrm_amount,date
22,00126380,005930,매출액,OFS,39.729088,2023-06-30
17,00126380,005930,비유동부채,OFS,29.536245,2023-06-30
24,00126380,005930,법인세차감전 순이익,OFS,9.468528,2023-06-30
12,00126380,005930,당기순이익,CFS,1.723571,2023-06-30
21,00126380,005930,자본총계,OFS,218.188792,2023-06-30


In [16]:
# 여러 년도, 보고서 데이터를 부르기 위한 함수를 정의한다.
def SingleCorporate(KEY, CODE, YEARS, REPORTS):
    total = pd.DataFrame()
    for y in YEARS:
        for r in REPORTS:
            url = "https://opendart.fss.or.kr/api/fnlttSinglAcnt.json"
            res = requests.get(url, params={'crtfc_key': KEY, 'corp_code': CODE, 'bsns_year': y, 'reprt_code': r})
            data = json.loads(res.text)
            single = json_normalize(data['list']) [['reprt_code','bsns_year','corp_code', 'stock_code', 'account_nm', 'fs_div', 'thstrm_amount']]
            # 데이터 전처리
            single['monday'] = single['reprt_code'].map({'11011':'1231','11012':'0630','11013':'0331','11014':'0931'})
            single['date'] = pd.to_datetime(single['bsns_year']+single['monday'], format='%Y%m%d', errors='coerce')
            single.drop(['reprt_code','bsns_year','monday'], axis=1, inplace=True)
            single['thstrm_amount'] = single['thstrm_amount'].str.replace(',', '').astype(float)/1000000000000
            total = pd.concat([total, single])
    return total.reset_index(drop=True)

#### 4. 상장기업 단일회사 전체 정보
- [API설명서] https://opendart.fss.or.kr/guide/detail.do?apiGrpCd=DS003&apiId=2019020

In [17]:
# corp_code, bsns_year, reprt_code, fs_div는 필수로 입력되어야 한다.
code = '00126380'
year = 2023
report = 11012   # 1분기(11013, 0331), 반기(11012, 0630), 3분기(11014, 0931), 사업(11011, 1231)
fs = 'OFS'       # CFS(연결재무제표), OFS(재무제표)

url = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json"
res = requests.get(url, params={'crtfc_key': key, 'corp_code': code, 'bsns_year': year, 'reprt_code': report, 'fs_div': fs})
data = json.loads(res.text)

In [18]:
# 기업코드, 년도, 재무제표 구분, 계정명, 당기금액
data = json_normalize(data['list']) [["corp_code", "bsns_year","sj_div", "account_nm", "thstrm_amount"]]
display(data.sample(5))

,corp_code,bsns_year,sj_div,account_nm,thstrm_amount
111,00126380,2023,SCE,기말자본,4403893000000
89,00126380,2023,CF,배당금의 지급,4905792000000
4,00126380,2023,BS,미수금,2041832000000
109,00126380,2023,SCE,기말자본,213225614000000
0,00126380,2023,BS,유동자산,73472493000000
